<a href="https://www.kaggle.com/code/cyberkarim/plant-seedling-classification-project-experiments?scriptVersionId=214909767" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers
import keras
import random
import numpy as np
%matplotlib inline
import tqdm
import cv2
import os 
#import tensorflow_addons as tfa
from keras import layers
import gc
from tensorflow.keras.regularizers import l2
import keras_cv
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import load_model
import seaborn as sns
import os

# Image preprocessing 

In [ ]:
pwd = "/kaggle/input/v2-plant-seedlings-dataset/"
training_images_files_names_black_grass = len(os.listdir(pwd+"Black-grass"))
training_images_files_names_Charlock = len(os.listdir(pwd+"Charlock"))
training_images_files_names_Cleavers = len(os.listdir(pwd+"Cleavers"))
training_images_files_names_Common_Chickweed = len(os.listdir(pwd+"Common Chickweed"))
training_images_files_names_Common_wheat = len(os.listdir(pwd+"Common wheat"))
training_images_files_names_Scentless_Mayweed = len(os.listdir(pwd+"Scentless Mayweed"))
training_images_files_names_Fat_Hen = len(os.listdir(pwd+"Fat Hen"))
training_images_files_names_Loose_Silky_bent = len(os.listdir(pwd+"Loose Silky-bent"))
training_images_files_names_Maize = len(os.listdir(pwd+"Maize"))
training_images_files_names_Shepherds_Purse = len(os.listdir(pwd+"Shepherd’s Purse"))
training_images_files_names_Small_flowered_Cranesbill = len(os.listdir(pwd+"Small-flowered Cranesbill"))
training_images_files_names_Sugar_beet = len(os.listdir(pwd+"Sugar beet"))
print("black grass sample size : ",training_images_files_names_black_grass,"\n")
print("Charlock sample size : ",training_images_files_names_Charlock,"\n")
print("Cleavers sample size : ",training_images_files_names_Cleavers,"\n")
print("Common Chickweed sample size : ",training_images_files_names_Common_Chickweed,"\n")
print("Common wheat sample size : ",training_images_files_names_Common_wheat,"\n")
print("Scentless Mayweed sample size : ",training_images_files_names_Scentless_Mayweed,"\n")
print("Fat Hen sample size : ",training_images_files_names_Fat_Hen,"\n")
print("Loose Silky sample size : ",training_images_files_names_Loose_Silky_bent,"\n")
print("Maize sample size : ",training_images_files_names_Maize,"\n")
print("Shepherds Purse sample size : ",training_images_files_names_Shepherds_Purse,"\n")
print("Small flowered Cranesbill sample size : ",training_images_files_names_Small_flowered_Cranesbill,"\n")
print("Sugar_beet sample size : ",training_images_files_names_Sugar_beet,"\n")
occurences = [training_images_files_names_black_grass/4750,training_images_files_names_Charlock/4750,training_images_files_names_Cleavers/4750,training_images_files_names_Common_Chickweed/4750,training_images_files_names_Common_wheat/4750,training_images_files_names_Fat_Hen/4750,training_images_files_names_Loose_Silky_bent/4750,training_images_files_names_Maize/4750,training_images_files_names_Scentless_Mayweed/4750,training_images_files_names_Shepherds_Purse/4750,training_images_files_names_Small_flowered_Cranesbill/4750,training_images_files_names_Sugar_beet/4750]
fig = plt.figure(figsize=[20,10])
ax = fig.add_axes([0,0,1,1])
ax.bar(["Black-grass","Charlock","Cleavers","Common Chickweed","Common wheat","Fat Hen","Loose Silky-bent","Maize","Scentless Mayweed","Shepherds Purse","Small-flowered Cranesbill","Sugar beet"],occurences)
plt.show()



In [ ]:
label_encoding_mapper = {"Black-grass" : 0,"Charlock" : 1,"Cleavers": 2,"Common Chickweed" : 3,"Common wheat" : 4,"Fat Hen" : 5,"Loose Silky-bent" : 6,"Maize" : 7,"Scentless Mayweed" : 8,"Shepherd’s Purse" : 9,"Small-flowered Cranesbill" : 10,"Sugar beet" : 11}
labels = ["Black-grass","Charlock","Cleavers","Common Chickweed","Common wheat","Fat Hen","Loose Silky-bent","Maize","Scentless Mayweed","Shepherd’s Purse","Small-flowered Cranesbill","Sugar beet"]
training_size = training_images_files_names_black_grass + training_images_files_names_Charlock + training_images_files_names_Cleavers + training_images_files_names_Common_Chickweed + training_images_files_names_Common_wheat + training_images_files_names_Fat_Hen + training_images_files_names_Loose_Silky_bent + training_images_files_names_Maize + training_images_files_names_Scentless_Mayweed + training_images_files_names_Shepherds_Purse + training_images_files_names_Small_flowered_Cranesbill + training_images_files_names_Sugar_beet
y_train =  np.array([[0 for i in range(0,12)] for k in range(0,training_size)])
plant_species = os.listdir("/kaggle/input/v2-plant-seedlings-dataset")
x_train = []
index = 0
for plant_specie in plant_species:
   images_file_name = os.listdir(pwd+plant_specie)
   for i in range(0,len(images_file_name)):
     img = cv2.imread(pwd+plant_specie+"/"+images_file_name[i], cv2.IMREAD_COLOR)
     if plant_specie not in labels:
         continue
     img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
     #if img.shape != (100,100,3):
     resized = cv2.resize(img,(100,100), interpolation = cv2.INTER_AREA)
     x_train = x_train + [resized]
     y_train[index][label_encoding_mapper[plant_specie]] = 1 
     index = index + 1
x_train = np.array(x_train)
y_train = np.array(y_train)


In [ ]:
print("training sample : ",x_train.shape,"  ",y_train.shape,"\n")



In [ ]:
plant_species= ["Black-grass","Charlock","Cleavers","Common Chickweed","Common wheat","Fat Hen","Loose Silky-bent","Maize","Scentless Mayweed","Shepherd’s Purse","Small-flowered Cranesbill","Sugar beet"]
index_storage = {"Black-grass" : [],"Charlock" : [],"Cleavers": [],"Common Chickweed" : [],"Common wheat" : [],"Fat Hen" : [],"Loose Silky-bent" : [],"Maize" : [],"Scentless Mayweed" : [],"Shepherd’s Purse" : [],"Small-flowered Cranesbill" : [],"Sugar beet" : []}
for i in range(0,len(x_train)):
    index = y_train[i].tolist().index(1)
    index_storage[plant_species[index]] = index_storage[plant_species[index]] + [i]

In [ ]:
plant_species = os.listdir("/kaggle/input/v2-plant-seedlings-dataset")
for plant_specie in plant_species:
   images_file_name = os.listdir(pwd+plant_specie)
   sample_size_to_display = random.sample(range(0,100), 3)
   if plant_specie not in labels:
     print("yes")
     continue
   print("plant species :",plant_specie)
   for i in range(0,3): 
     img = cv2.imread(pwd+plant_specie+"/"+images_file_name[sample_size_to_display[i]], cv2.IMREAD_COLOR)
     img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
     plt.imshow(img)
     plt.show()

# Image processing

In [ ]:
def segment_image(img):
  img1 = cv2.GaussianBlur(img,(7, 7), 0) 
  img2 = cv2.cvtColor(np.float32(img1),cv2.COLOR_RGB2HSV)
  green_segment_values = 50
  hsv_lower_bound = np.array([100-green_segment_values,0,0])
  hsv_upper_bound = np.array([100+green_segment_values,255,240])
  mask = cv2.inRange(img2,hsv_lower_bound,hsv_upper_bound)
  kernel = np.ones((3,3),np.uint8)
  mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
  return mask

  

In [ ]:
def preprocess_dataset(x_train):
  preprocessed_data = []
  for i in range(0,x_train.shape[0]):
    preprocessed_data = preprocessed_data + [segment_image(x_train[i])]
  return preprocessed_data 

In [ ]:
preprocessed_data = np.array(preprocess_dataset(x_train))

#vizualize_random_images(x_train,preprocessed_data,10)

In [ ]:
(x_training, valX, y_training, valY) = train_test_split(x_train,y_train,test_size=0.15, random_state=10)


In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,horizontal_flip=True,vertical_flip=True,rescale=1/255)
datagen.fit(x_training)
it = datagen.flow(x_training,y_training,batch_size=32)
datagenval = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
datagenval.fit(valX)
itval = datagenval.flow(valX, valY)

In [ ]:
(x_training_processed, valX_processed, y_training_processed, valY_processed) = train_test_split(preprocessed_data,y_train,test_size=0.15, random_state=10)

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,horizontal_flip=True,vertical_flip=True)
x_training_processed = tf.reshape(x_training_processed,shape=(4708, 100, 100,1))
datagen.fit(x_training_processed)
training_processed_data_flow = datagen.flow(x_training_processed,y_training_processed)
datagenval = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
valX_processed = tf.reshape(valX_processed,shape=(831, 100, 100,1))
datagenval.fit(valX_processed)
validation_processed_data_flow = datagenval.flow(valX_processed,valY_processed)

In [ ]:
INIT_LR = 1e-2 # initial learning rate
BATCH_SIZE = 32
EPOCHS = 5

s = tf.keras.backend.clear_session()  # clear default graph
# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
#model = make_model()  # define our model
#learning_rate=INIT_LR
opt = tf.keras.optimizers.Adam(learning_rate=INIT_LR)
# prepare model for fitting (loss, optimizer, etc)
tqdm_callback = tf.keras.callbacks.ProgbarLogger()



# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
     #return (INIT_LR*0.9)**epoch
   return (INIT_LR)  
    #return INIT_LR

class MetricsAndLearningRatePrinter(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Retrieve the learning rate from the optimizer
        lr = self.model.optimizer.learning_rate
        if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule):
            lr = lr(epoch)
        lr_value = lr.numpy()
        
        # Extract metrics from logs
        train_loss = logs.get('loss', 'N/A')
        val_loss = logs.get('val_loss', 'N/A')
        train_f1 = logs["f1_score"]
        val_f1 = logs["val_f1_score"]
        
        # Print all metrics and learning rate
        print(f"Epoch {epoch + 1}:")
        print(f"learning rate {lr_value + 1}:")
        print("  - Training Loss: ",train_loss)
        print("  - Training f1 score: ",train_f1)
        print("  - Validation Loss: ",val_loss)
        print("  - Validation f1 score: ",val_f1)

class LearningRatePrinter(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Retrieve the learning rate from the optimizer
        lr = self.model.optimizer.learning_rate
        # If the learning rate is a schedule, call it with the current epoch
        if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule):
            lr = lr(epoch)
        print(f"Epoch {epoch + 1}: Learning Rate = {lr.numpy():.6f}")

class GarbageCollectorCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()


# Modeling

In [ ]:
def make_customized_cnn_model(image_number_channel):
    

    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides = (1, 1), padding="same", name = 'conv1', input_shape=(100,100,image_number_channel)))   
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    

    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides = (1, 1), padding="same", name = 'conv2'))   
    model.add(tf.keras.layers.MaxPooling2D((2, 2),padding="same",name='max_pool_1'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), strides = (1, 1), padding="same", name = 'conv3'))   
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
   
    model.add(tf.keras.layers.Conv2D(128, (3, 3), strides = (1, 1), padding="same", name = 'conv4'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2),padding="same",name='max_pool_2'))   
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))
    
    
    model.add(tf.keras.layers.Conv2D(256, (3, 3), strides = (1, 1), padding="same", name = 'conv5', input_shape=(100,100,1)))   
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
   
    model.add(tf.keras.layers.Conv2D(256, (3, 3), strides = (1, 1), padding="same", name = 'conv6', input_shape=(100,100,1))) 
    model.add(tf.keras.layers.MaxPooling2D((2, 2),padding="same",name='max_pool_3'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    
    

    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))

    model.add(tf.keras.layers.Flatten())    
    
    
    model.add(tf.keras.layers.Dense(256, name='fc1'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))
    model.add(tf.keras.layers.Dense(256, name='fc2'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.10, noise_shape=None, seed=0))

    model.add(tf.keras.layers.Dense(12))    
    model.add(tf.keras.layers.Activation("softmax"))  
    return model

In [ ]:
s = tf.keras.backend.clear_session()
model = make_customized_cnn_model(1)
opt = tf.keras.optimizers.Adam(learning_rate=INIT_LR)

metrics_lr_printer = MetricsAndLearningRatePrinter()
model.summary()

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,  # for SGD
    metrics=[tf.keras.metrics.F1Score(average="micro")]  # report accuracy during training
)


# Save the weights using the `checkpoint_path` format

#(valX_processed,valY_processed)
history = model.fit(
    training_processed_data_flow,  # prepared data
    epochs=200,
    validation_data=(valX_processed,valY_processed),
    steps_per_epoch=len(x_training) // BATCH_SIZE,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               tqdm_callback,
               GarbageCollectorCallback(),
               metrics_lr_printer
               ],
    shuffle=True
    ,verbose=0)

model.save('modelcnn1.keras')
best_score = max(history.history['val_f1_score'])
print("best f1 score on validation data :",best_score)
plt.plot(history.history['loss'],"-b",label="training data loss")
plt.plot(history.history['val_loss'],"-r",label="validation data loss")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show() 

In [ ]:
s = tf.keras.backend.clear_session()
model = make_customized_cnn_model(3)
opt = tf.keras.optimizers.Adam(learning_rate=INIT_LR)

metrics_lr_printer = MetricsAndLearningRatePrinter()
model.summary()


model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,  # for SGD
    metrics=[tf.keras.metrics.F1Score(average="micro")]  # report accuracy during training
)
history = model.fit(
    it,  
    epochs=200,
    validation_data=itval,
    steps_per_epoch=len(x_training) // BATCH_SIZE,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               tf.keras.callbacks.ModelCheckpoint("/kaggle/working/modelcnn3.keras",save_best_only=True,monitor="val_f1_score"),
               tqdm_callback,
               GarbageCollectorCallback(),
               metrics_lr_printer
               ],
    shuffle=True
    ,verbose=0)


model.save('modelcnn3.keras')

best_score = max(history.history['val_f1_score'])
print("best f1 score on validation data :",best_score)
plt.plot(history.history['loss'],"-b",label="training loss")
plt.plot(history.history['val_loss'],"-r",label="validation loss")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show() 

# Model evaluation

## RGB images model

In [ ]:
cnnmodel3= tf.keras.models.load_model('/kaggle/working/modelcnn3.keras')


In [ ]:
y_pred_test = cnnmodel3.predict(valX/255)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)

In [ ]:
results = cnnmodel3.evaluate(valX/255,valY)
print("val loss, val f1 score:", results)


In [ ]:
cm = confusion_matrix(np.argmax(valY,axis=1),y_pred_test_classes)

# Optional: Print Classification Report
print(classification_report(np.argmax(valY,axis=1), y_pred_test_classes))

# Step 3: Visualize Confusion Matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=True, yticklabels=True)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

## Segmented image model

In [ ]:
cnnmodel1 = tf.keras.models.load_model("/kaggle/working/modelcnn1.keras")
#cnnmodel1.load_weights("/kaggle/working/modelcnn1.weights.h5")




In [ ]:
y_pred_test = cnnmodel1.predict(valX_processed)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)

In [ ]:
results = cnnmodel1.evaluate(valX_processed,valY_processed)
print("test loss, test acc:", results)


In [ ]:
cm = confusion_matrix(np.argmax(valY_processed,axis=1),y_pred_test_classes)

# Optional: Print Classification Report
print(classification_report(np.argmax(valY_processed,axis=1), y_pred_test_classes))

# Step 3: Visualize Confusion Matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=True, yticklabels=True)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()